[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://githubtocolab.com/Satyajeet-code/Generative-AI/blob/main/RAGSession42AI/HybridSearchFile2.ipynb)

In [3]:
!pip -q install langchain rank_bm25 faiss-cpu langchain-google-genai langchain-groq  langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.3 MB/s eta 0:00:00


In [20]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever
# from langchain.schema import Document

# from langchain.vectorstores import Chroma
from langchain.vectorstores import FAISS
import google.generativeai as genai
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.embeddings import HuggingFaceBgeEmbeddings

genai.configure(api_key="")

### Trying out BM25

In [5]:
documents = [
    "satyajeet is taking a session",
    "everybody is attending the session",
    "this is on rag",
    "session also covers advanced rag"
]

In [9]:
bm25_retriever = BM25Retriever.from_texts(documents)
bm25_retriever.k = 3

In [10]:
bm25_retriever.invoke("satyajeet")

[Document(metadata={}, page_content='satyajeet is taking a session'),
 Document(metadata={}, page_content='session also covers advanced rag'),
 Document(metadata={}, page_content='this is on rag')]

### Creating dense retriever

In [21]:
# embedding = GoogleGenerativeAIEmbeddings(model = "models/embedding-001")
model_name="BAAI/bge-base-en-v1.5"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}
embedding = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [23]:
faiss_vectorstore = FAISS.from_texts(documents, embedding)
faiss_retriever = faiss_vectorstore.as_retriever(search_kwargs={"k": 2})

In [24]:
faiss_retriever.get_relevant_documents("who is attending the session")

[Document(metadata={}, page_content='everybody is attending the session'),
 Document(metadata={}, page_content='satyajeet is taking a session')]

In [25]:
ensemble_retriever = EnsembleRetriever(retrievers=[bm25_retriever, faiss_retriever],
                                       weights=[0.5, 0.5])

In [27]:
docs = ensemble_retriever.get_relevant_documents("what does the session cover")
docs

[Document(metadata={}, page_content='everybody is attending the session'),
 Document(metadata={}, page_content='session also covers advanced rag'),
 Document(metadata={}, page_content='satyajeet is taking a session')]

In [28]:
faiss_retriever.get_relevant_documents("what does the session cover")

[Document(metadata={}, page_content='session also covers advanced rag'),
 Document(metadata={}, page_content='everybody is attending the session')]

In [29]:
bm25_retriever.invoke("what does the session cover")

[Document(metadata={}, page_content='everybody is attending the session'),
 Document(metadata={}, page_content='session also covers advanced rag'),
 Document(metadata={}, page_content='satyajeet is taking a session')]